In [1]:
## Data Collection
import nltk 
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## Load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')
## save to a file
with open('hamlet.txt', 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/abhishekkumar/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Data Preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow .keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## load the dataset
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

## Tokenize the text-creating indexes for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words =len(tokenizer.word_index) + 1
total_words

In [ ]:
tokenizer.word_index

In [ ]:
## create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
input_sequences

In [ ]:
## Pad Sequences
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

In [ ]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

In [ ]:
##Create predictors and label
import tensorflow as tf

x, y = input_sequences[:,:-1], input_sequences[:,-1]

In [ ]:
x

In [ ]:
y

In [ ]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
y

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 1, restore_best_weights = True)

In [ ]:
# Train our LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

## Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

## Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
## Train the model
history = model.fit(x_train, y_train, epochs=100, verbose=1, validation_data=(x_test, y_test)) #, callbacks = [early_stopping] )

In [ ]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  # Ensure the sequence length matches max_sequence_len-1
    token_list = pad_sequences([token_list], maxlen = max_sequence_len-1, padding = 'pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [ ]:
input_text = "To be or not to be"
print(f"Input text:{input_text}")
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction:{next_word}")

In [ ]:
## Save the model
model.save('next_word_lstm.h5')
## Save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)